In [ ]:
from flask import Flask, request, jsonify, render_template
import pandas as pd
import joblib

In [7]:
data = pd.read_csv('supermarket_data.csv')

In [8]:

def recommend_for_user(model, test_data, user_id, num_recs):
    # Get the user's interactions from the test dataset
    user_interactions = test_data[test_data['User ID'] == user_id]
    
    # Generate top-k recommendations for the user
    top_k = model.recommend_k_items(user_interactions, top_k=num_recs, remove_seen=True)
    
    # Join with product names
    top_k_with_titles = top_k.join(data[['Product ID', 'Product Name']].drop_duplicates().set_index('Product ID'), 
                                    on='Product ID', 
                                    how='inner').sort_values(by='Prediction', ascending=False)
    
    # Return the top recommendations for the user
    return top_k_with_titles.head(num_recs)


In [9]:
app = Flask(__name__)
model = joblib.load('model.joblib')
test = pd.read_csv('test_data.csv')

@app.route('/', methods=['GET'])
def index():
    return render_template('index.html')


# Define the API endpoint
@app.route('/recommendations', methods=['GET'])
def get_recommendations():
    # Get the user ID and number of recommendations from the query parameters
    user_id = int(request.args.get('user_id'))
    
    # Generate recommendations for the user
    recs = recommend_for_user(model, test, user_id, 10)
    recs_dict = recs.to_dict('records')
    return render_template('recommendations.html', recommendations=recs_dict, user_id=user_id)


    

if __name__ == '__main__':
    app.run(debug=True,use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [14/Apr/2023 08:25:33] "GET /recommendations?user_id=142 HTTP/1.1" 200 -
